In [3]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('/Users/sara/Documents/算法学习/NLP/nlp名企/作业讲课资料/1-2/seq2seq/code')

## 1. 数据预处理

In [5]:
import pandas as pd
import numpy as np
from utils.data_loader import build_dataset,pad_proc,sentences_proc
from utils.config import *
from utils.multi_proc_utils import parallelize
from gensim.models.word2vec import LineSentence, Word2Vec

## 1.1加载数据

In [ ]:
train_data_path
test_data_path

In [8]:
train_df = pd.read_csv(train_data_path)
test_df = pd.read_csv(test_data_path)
print('train data size {},test data size {}'.format(len(train_df), len(test_df)))

train data size 82943,test data size 20000


In [9]:
train_df['Dialogue'].head()

0    技师说：[语音]|车主说：新的都换了|车主说：助力泵，方向机|技师说：[语音]|车主说：换了...
1    技师说：你这个有没有电脑检测故障代码。|车主说：有|技师说：发一下|车主说：发动机之前亮故障...
2    技师说：你好，4缸自然吸气发动机N46是吧，先挂空档再挂其他档有没有闯动呢，变速箱油液位是否...
3    技师说：右侧排气管上方，缸体上靠近变速箱|车主说：[图片]|车主说：是不是这个？|车主说：这...
4    技师说：家庭用车的话，还是可以入手的|技师说：维修保养费用不高|车主说：12年的180市场价...
Name: Dialogue, dtype: object

In [33]:
train_df['Dialogue'][0]

'技师说：[语音]|车主说：新的都换了|车主说：助力泵，方向机|技师说：[语音]|车主说：换了方向机带的有|车主说：[图片]|技师说：[语音]|车主说：有助力就是重，这车要匹配吧|技师说：不需要|技师说：你这是更换的部件有问题|车主说：跑快了还好点，就倒车重的很。|技师说：是非常重吗|车主说：是的，累人|技师说：[语音]|车主说：我觉得也是，可是车主是以前没这么重，选吧助理泵换了不行，又把放向机换了，现在还这样就不知道咋和车主解释。|技师说：[语音]|技师说：[语音]'

In [34]:
test_df.head()

,QID,Brand,Model,Question,Dialogue
0,Q1,大众(进口),高尔夫(进口),我的帕萨特烧机油怎么办怎么办？,技师说：你好，请问你的车跑了多少公里了，如果在保修期内，可以到当地的4店里面进行检查维修。如...
1,Q2,一汽-大众奥迪,奥迪A6,修一下多少钱是换还是修,技师说：你好师傅！抛光处理一下就好了！50元左右就好了，希望能够帮到你！祝你生活愉快！
2,Q3,上汽大众,帕萨特,帕萨特领域 喇叭坏了 店里说方向盘里线坏了 换一根两三百不等 感觉太贵,技师说：你好，气囊油丝坏了吗，这个价格不贵。可以更换。
3,Q4,南京菲亚特,派力奥,发动机漏气会有什么征兆？,技师说：你好！一：发动机没力，并伴有“啪啪”的漏气声音。二：发动机没力，并伴有排气管冒黑烟。...
4,Q5,东风本田,思铂睿,请问 那天右后胎扎了订，补了胎后跑高速80多开始有点抖，110时速以上抖动明显，以为是未做动...,技师说：你好师傅！可能前轮平衡快脱落或者不平衡造成的！建议前轮做一下动平衡就好了！希望能够帮...


In [35]:
train_df.head()

,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机重，助力泵，方向机都换了还是一样,技师说：[语音]|车主说：新的都换了|车主说：助力泵，方向机|技师说：[语音]|车主说：换了...,随时联系
1,Q2,奔驰,奔驰M级,奔驰ML500排气凸轮轴调节错误,技师说：你这个有没有电脑检测故障代码。|车主说：有|技师说：发一下|车主说：发动机之前亮故障...,随时联系
2,Q3,宝马,宝马X1(进口),2010款宝马X1，2011年出厂，2.0排量，通用6L45变速箱，原地换挡位PRND车辆闯...,技师说：你好，4缸自然吸气发动机N46是吧，先挂空档再挂其他档有没有闯动呢，变速箱油液位是否...,行驶没有顿挫的感觉，原地换挡有闯动，刹车踩重没有，这是力的限制的作用，应该没有问题
3,Q4,Jeep,牧马人,3.0V6发动机号在什么位置，有照片最好！,技师说：右侧排气管上方，缸体上靠近变速箱|车主说：[图片]|车主说：是不是这个？|车主说：这...,举起车辆，在左前轮这边的缸体上
4,Q5,奔驰,奔驰C级,2012款奔驰c180怎么样，维修保养，动力，值得拥有吗,技师说：家庭用车的话，还是可以入手的|技师说：维修保养费用不高|车主说：12年的180市场价...,家庭用车可以入手的，维修保养价格还可以。车况好，价格合理可以入手


## 1.2 空值填充

In [11]:
# 2. 空值剔除
train_df.dropna(subset=['Report'], inplace=True)

train_df.fillna('', inplace=True)
test_df.fillna('', inplace=True)

## 1.3.多核, 批量数据处理

In [12]:
%%time
train_df=sentences_proc(train_df)
test_df=sentences_proc(test_df)

CPU times: user 6min 10s, sys: 2.48 s, total: 6min 13s
Wall time: 6min 16s


In [13]:
%%time
train_df = parallelize(train_df, sentences_proc)
test_df = parallelize(test_df, sentences_proc)

CPU times: user 1.2 s, sys: 617 ms, total: 1.82 s
Wall time: 1min 24s


In [10]:
# %%time
# train_df = parallelize(train_df, sentences_proc)
# test_df = parallelize(test_df, sentences_proc)

## 1.4 合并训练测试数据

In [14]:
train_df['merged'] = train_df[['Question', 'Dialogue', 'Report']].apply(lambda x: ' '.join(x), axis=1)
test_df['merged'] = test_df[['Question', 'Dialogue']].apply(lambda x: ' '.join(x), axis=1)
merged_df = pd.concat([train_df[['merged']], test_df[['merged']]], axis=0)
print('train data size {},test data size {},merged_df data size {}'.format(len(train_df), len(test_df),len(merged_df)))

train data size 82873,test data size 20000,merged_df data size 102873


## 1.5 保存处理好的 训练 测试集合

In [15]:
train_df = train_df.drop(['merged'], axis=1)
test_df = test_df.drop(['merged'], axis=1)
train_df.to_csv(train_seg_path, index=None, header=True)
test_df.to_csv(test_seg_path, index=None, header=True)
merged_df.to_csv(merger_seg_path, index=None, header=False)

# 2. 词向量

## 2.1 预训练词向量

In [16]:
merger_seg_path

'/Users/sara/Documents/算法学习/NLP/nlp名企/作业讲课资料/1-2/seq2seq/code/data/merged_train_test_seg_data.csv'

In [17]:
wv_model = Word2Vec(LineSentence(merger_seg_path),
                    size=300, 
                    negative=5, 
                    workers=8,
                    iter=wv_train_epochs, 
                    window=3,
                    min_count=5)

2020-08-19 17:11:27,112 : INFO : collecting all words and their counts
2020-08-19 17:11:27,113 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-19 17:11:27,572 : INFO : PROGRESS: at sentence #10000, processed 1280513 words, keeping 33450 word types
2020-08-19 17:11:27,985 : INFO : PROGRESS: at sentence #20000, processed 2579769 words, keeping 48483 word types
2020-08-19 17:11:28,455 : INFO : PROGRESS: at sentence #30000, processed 3854040 words, keeping 59566 word types
2020-08-19 17:11:28,862 : INFO : PROGRESS: at sentence #40000, processed 5093606 words, keeping 68837 word types
2020-08-19 17:11:29,300 : INFO : PROGRESS: at sentence #50000, processed 6408013 words, keeping 77054 word types
2020-08-19 17:11:29,759 : INFO : PROGRESS: at sentence #60000, processed 7805959 words, keeping 85348 word types
2020-08-19 17:11:30,227 : INFO : PROGRESS: at sentence #70000, processed 9234104 words, keeping 93454 word types
2020-08-19 17:11:30,722 : INFO : PROGRE

In [19]:
wv_model

## 2.2. 建立词表

In [20]:
vocab = {word: index for index, word in enumerate(wv_model.wv.index2word)}
reverse_vocab = {index: word for index, word in enumerate(wv_model.wv.index2word)}
len(vocab)

31081

In [ ]:
vocab

In [21]:
sentence=['技师说','帕萨特','机油']

In [22]:
[vocab[word]for word in sentence]

[1, 1071, 24]

In [23]:
[reverse_vocab[index] for index in [1, 1076, 24]]

['技师说', '两边', '机油']

## Q 1 .使用了min_count,其实部分词不在vocab表中 ,但是训练数据和测试数据中又有这些词?

---


## 2.3. 获取词向量矩阵

In [24]:
embedding_matrix = wv_model.wv.vectors
embedding_matrix.shape

(31081, 300)

In [25]:
embedding_matrix

array([[ 1.8661356e-01,  2.6669255e-01, -2.0303488e-01, ...,
        -3.7490058e-01,  1.5657571e-01, -4.8082581e-01],
       [ 3.6642390e-01, -4.0947658e-01, -1.2689227e-01, ...,
         5.5439848e-02, -7.4810579e-02,  6.6715712e-03],
       [-4.1605449e-01,  6.8964325e-02,  5.3321660e-01, ...,
         2.4555589e-01,  3.1569359e-01, -3.8069963e-01],
       ...,
       [-3.5903111e-02,  1.5965585e-02,  3.8452737e-02, ...,
         8.1684219e-04, -2.0354069e-04, -1.8332239e-02],
       [ 4.0782291e-02,  3.3266533e-02, -4.6292227e-03, ...,
         8.7010236e-03,  4.5694578e-03,  4.6086729e-02],
       [ 8.6475909e-03, -1.6157508e-02,  3.1429492e-02, ...,
         3.0564722e-02,  5.9913035e-02,  2.0184094e-02]], dtype=float32)

## 3. 构建训练数据

+ 可以把Question,Dialogue当做一句 `长文本处理`, 合并构建成X
+ Report作为需要预测的标签,构建Y

In [26]:
train_df['X'] = train_df[['Question', 'Dialogue']].apply(lambda x: ' '.join(x), axis=1)
test_df['X'] = test_df[['Question', 'Dialogue']].apply(lambda x: ' '.join(x), axis=1)

In [27]:
train_df['X'].head()

0    方向机 重 ， 助力 泵 ， 方向机 都 换 技师说 车主说 新 都 换 车主说 助力 泵 ...
1    奔驰 ML500 排气 凸轮轴 调节 错误 技师说 有没有 电脑 检测 故障 代码 。 车主...
2    2010 款 宝马X1 ， 2011 年 出厂 ， 2.0 排量 ， 通用 6L45 变速箱...
3    3.0 V6 发动机 号 位置 ， 照片 最好 ！ 技师说 右侧 排气管 上方 ， 缸体 上...
4    2012 款 奔驰 c180 ， 维修保养 ， 动力 ， 值得 拥有 技师说 家庭 用车 ，...
Name: X, dtype: object

# Q 2. 句子长度一样 ? 如何构建训练,batch操作,矩阵 ...

---

## 3.1 填充字段

In [28]:
def pad_proc(sentence, max_len, vocab):
    '''
    < start > < end > < pad > < unk >
    '''
    # 0.按空格统计切分出词
    words = sentence.strip().split(' ')
    # 1. 截取规定长度的词数
    words = words[:max_len]
    # 2. 填充< unk > ,判断是否在vocab中, 不在填充 < unk >
    sentence = [word if word in vocab else '<UNK>' for word in words]
    # 3. 填充< start > < end >
    sentence = ['<START>'] + sentence + ['<STOP>']
    # 4. 判断长度，填充　< pad >
    sentence = sentence + ['<PAD>'] * (max_len + 2 - len(words))
    return ' '.join(sentence)

In [ ]:
pad_proc('技师说 帕萨特 机油', max_len=2, vocab=vocab)

# 休息一下，9.10分继续

# Q3. 如何确定max_len的值? 经验 ?

## 3.2 获取适当的Max_Len

In [30]:
def get_max_len(data):
    """
    获得合适的最大长度值
    :param data: 待统计的数据  train_df['Question']
    :return: 最大长度值
    """
    max_lens = data.apply(lambda x: x.count(' '))
    return int(np.mean(max_lens) + 2 * np.std(max_lens))

In [31]:
# 获取输入数据 适当的最大长度
train_y_max_len = get_max_len(train_df['X'])
test_y_max_len = get_max_len(test_df['X'])
y_max_len = max(train_y_max_len, test_y_max_len)

# 获取标签数据 适当的最大长度
train_y_max_len = get_max_len(train_df['Report'])

In [32]:
train_y_max_len

38

In [34]:
X_max_len=200

## 3.3 填充处理

+ < start > - 句子开始
+ < end > - 句子结尾
+ < pad > - 短句填充
+ < unk > - 未知词

In [35]:
# 训练集X处理
train_df['X'] = train_df['X'].apply(lambda x: pad_proc(x, X_max_len, vocab))
# 训练集Y处理
train_df['Y'] = train_df['Report'].apply(lambda x: pad_proc(x, train_y_max_len, vocab))
# 测试集X处理
test_df['X'] = test_df['X'].apply(lambda x: pad_proc(x, X_max_len, vocab))

In [36]:
train_x_pad_path

'/Users/sara/Documents/算法学习/NLP/nlp名企/作业讲课资料/1-2/seq2seq/code/data/train_X_pad_data.csv'

In [37]:
# 保存中间结果数据
train_df['X'].to_csv(train_x_pad_path, index=None, header=False)
train_df['Y'].to_csv(train_y_pad_path, index=None, header=False)
test_df['X'].to_csv(test_x_pad_path, index=None, header=False)

In [39]:
test_df['X']

0        <START> 帕萨特 烧 机油 ？ 技师说 你好 ， 请问 车 跑 公里 ， 保修期 内 ...
1        <START> 修 一下 钱 换 修 技师说 你好 师傅 ！ 抛光 处理 一下 好 ！ 50...
2        <START> 帕萨特 领域 喇叭 坏 店里 说 方向盘 <UNK> 坏 换 一根 两三百 ...
3        <START> 发动机 漏气 会 征兆 ？ 技师说 你好 ！ 发动机 没力 ， 伴有 “ 啪...
4        <START> 请问 那天 右 后 胎扎 订 ， 补 胎后 跑 高速 80 有点 抖 ， 1...
                               ...                        
19995    <START> 路虎 极光 2.0 t 发动机 ， 进气 链轮 损坏 更换 进气 链轮 后 ...
19996    <START> 别克 英朗XT <UNK> 后 ， 发现 左 雾灯 下雨 后 起雾 ， 更换...
19997    <START> 师傅 ， 想 问 下车 一周 开 两次 周一 开 过来 周五 开回去 对车 ...
19998    <START> 你好 大师 ， 车前 分泵 ， 制动 钳 导管 之间 晃动 ， 颠簸 路段 ...
19999    <START> 大师 好 ， 东风风光330 开车 下坡 时 没有 踩 刹车 四档 不 小心...
Name: X, Length: 20000, dtype: object

## Q4 新加的符号不在词表 和 词向量矩阵中,怎么办?

## 3.4 词表更新

In [40]:
print('start retrain w2v model')
wv_model.build_vocab(LineSentence(train_x_pad_path), update=True)
wv_model.train(LineSentence(train_x_pad_path), epochs=wv_train_epochs, total_examples=wv_model.corpus_count)
print('1/3')
wv_model.build_vocab(LineSentence(train_y_pad_path), update=True)
wv_model.train(LineSentence(train_y_pad_path), epochs=wv_train_epochs, total_examples=wv_model.corpus_count)
print('2/3')
wv_model.build_vocab(LineSentence(test_x_pad_path), update=True)
wv_model.train(LineSentence(test_x_pad_path), epochs=wv_train_epochs, total_examples=wv_model.corpus_count)

2020-08-19 17:15:10,431 : INFO : collecting all words and their counts
2020-08-19 17:15:10,432 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


start retrain w2v model


2020-08-19 17:15:11,089 : INFO : PROGRESS: at sentence #10000, processed 2040000 words, keeping 20639 word types
2020-08-19 17:15:11,708 : INFO : PROGRESS: at sentence #20000, processed 4080000 words, keeping 25440 word types
2020-08-19 17:15:12,316 : INFO : PROGRESS: at sentence #30000, processed 6120000 words, keeping 27774 word types
2020-08-19 17:15:12,925 : INFO : PROGRESS: at sentence #40000, processed 8160000 words, keeping 29070 word types
2020-08-19 17:15:13,516 : INFO : PROGRESS: at sentence #50000, processed 10200000 words, keeping 29847 word types
2020-08-19 17:15:14,050 : INFO : PROGRESS: at sentence #60000, processed 12240000 words, keeping 30394 word types
2020-08-19 17:15:14,606 : INFO : PROGRESS: at sentence #70000, processed 14280000 words, keeping 30695 word types
2020-08-19 17:15:15,140 : INFO : PROGRESS: at sentence #80000, processed 16320000 words, keeping 30840 word types
2020-08-19 17:15:15,308 : INFO : collected 30872 word types from a corpus of 16906092 raw wo

1/3


2020-08-19 17:16:06,378 : INFO : PROGRESS: at sentence #20000, processed 840000 words, keeping 10105 word types
2020-08-19 17:16:06,529 : INFO : PROGRESS: at sentence #30000, processed 1260000 words, keeping 11885 word types
2020-08-19 17:16:06,671 : INFO : PROGRESS: at sentence #40000, processed 1680000 words, keeping 13310 word types
2020-08-19 17:16:06,870 : INFO : PROGRESS: at sentence #50000, processed 2100000 words, keeping 14433 word types
2020-08-19 17:16:07,021 : INFO : PROGRESS: at sentence #60000, processed 2520000 words, keeping 15476 word types
2020-08-19 17:16:07,164 : INFO : PROGRESS: at sentence #70000, processed 2940000 words, keeping 16492 word types
2020-08-19 17:16:07,304 : INFO : PROGRESS: at sentence #80000, processed 3360000 words, keeping 17214 word types
2020-08-19 17:16:07,348 : INFO : collected 17434 word types from a corpus of 3480666 raw words and 82873 sentences
2020-08-19 17:16:07,349 : INFO : Updating model with new vocabulary
2020-08-19 17:16:07,363 : I

2/3


2020-08-19 17:16:19,878 : INFO : PROGRESS: at sentence #10000, processed 2040000 words, keeping 20674 word types
2020-08-19 17:16:20,423 : INFO : collected 25150 word types from a corpus of 4080000 raw words and 20000 sentences
2020-08-19 17:16:20,424 : INFO : Updating model with new vocabulary
2020-08-19 17:16:20,442 : INFO : New added 11395 unique words (31% of original 36545) and increased the count of 11395 pre-existing words (31% of original 36545)
2020-08-19 17:16:20,531 : INFO : deleting the raw counts dictionary of 25150 items
2020-08-19 17:16:20,533 : INFO : sample=0.001 downsamples 36 most-common words
2020-08-19 17:16:20,534 : INFO : downsampling leaves estimated 3297882 word corpus (81.4% of prior 4051742)
2020-08-19 17:16:20,593 : INFO : estimated required memory for 22790 words and 300 dimensions: 66091000 bytes
2020-08-19 17:16:20,593 : INFO : updating layer weights
2020-08-19 17:16:20,631 : WARNING : Effective 'alpha' higher than previous training cycles
2020-08-19 17:1

(8386644, 20400000)

In [41]:
# 保存词向量模型
wv_model.save(save_wv_model_path)

2020-08-19 17:16:32,699 : INFO : saving Word2Vec object under /Users/sara/Documents/算法学习/NLP/nlp名企/作业讲课资料/1-2/seq2seq/code/data/wv/word2vec.model, separately None
2020-08-19 17:16:32,701 : INFO : not storing attribute vectors_norm
2020-08-19 17:16:32,702 : INFO : not storing attribute cum_table
2020-08-19 17:16:33,637 : INFO : saved /Users/sara/Documents/算法学习/NLP/nlp名企/作业讲课资料/1-2/seq2seq/code/data/wv/word2vec.model


## Q5.为什么不一开始就添加 标志符号,然后训练词向量?

In [42]:
# 更新vocab 
vocab = {word: index for index, word in enumerate(wv_model.wv.index2word)}
reverse_vocab = {index: word for index, word in enumerate(wv_model.wv.index2word)}
# 更新词向量矩阵
embedding_matrix = wv_model.wv.vectors
embedding_matrix.shape

(31085, 300)

In [43]:
list(vocab.keys())[-5:]

['短发', '<START>', '<STOP>', '<PAD>', '<UNK>']

In [44]:
(31851, 300)

(31851, 300)

## Q6. 词可以训练吗?

## 3.4 数值转换

In [47]:
# 遇到未知词就填充unk的索引
unk_index = vocab['<UNK>']
def transform_data(sentence,vocab):
    # 字符串切分成词
    words=sentence.split(' ')
    # 按照vocab的index进行转换
    ids=[vocab[word] if word in vocab else unk_index for word in words]
    return ids

In [49]:
train_X=train_df['X']
train_Y=train_df['Y']
test_X=test_df['X']

In [50]:
train_X.head()

0    <START> 方向机 重 ， 助力 泵 ， 方向机 都 换 技师说 车主说 新 都 换 车...
1    <START> 奔驰 <UNK> 排气 凸轮轴 调节 错误 技师说 有没有 电脑 检测 故障...
2    <START> 2010 款 宝马X1 ， 2011 年 出厂 ， 2.0 排量 ， 通用 ...
3    <START> 3.0 V6 发动机 号 位置 ， 照片 最好 ！ 技师说 右侧 排气管 上...
4    <START> 2012 款 奔驰 c180 ， 维修保养 ， 动力 ， 值得 拥有 技师说...
Name: X, dtype: object

In [51]:
# 将词转换成索引  [<START> 方向机 重 ...] -> [32800, 403, 986, 246, 231
train_ids_x=train_X.apply(lambda x:transform_data(x,vocab))
train_ids_y=train_Y.apply(lambda x:transform_data(x,vocab))
test_ids_x=test_X.apply(lambda x:transform_data(x,vocab))

In [52]:
# 将索引列表转换成矩阵 [32800, 403, 986, 246, 231] --> array([[32800,   403,   986 ]]
train_data_X=np.array(train_ids_x.tolist())
train_data_Y=np.array(train_ids_y.tolist())
test_data_X=np.array(test_ids_x.tolist())

In [53]:
train_data_X.shape

(82873, 204)

In [54]:
train_data_X

array([[31081,   418,   802, ..., 31083, 31083, 31083],
       [31081,   815, 31084, ..., 31083, 31083, 31083],
       [31081,  1387,    59, ..., 31082, 31083, 31083],
       ...,
       [31081,   230,   898, ..., 31083, 31083, 31083],
       [31081, 12409,  3104, ..., 31083, 31083, 31083],
       [31081,    10,  3235, ..., 31083, 31083, 31083]])

## 4. 简易模型搭建

In [55]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [56]:
def seq2seq(input_length, output_sequence_length, embedding_matrix, vocab_size):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=300, weights=[embedding_matrix], trainable=False,
                        input_length=input_length))
    model.add(Bidirectional(GRU(300, return_sequences=False)))
    model.add(Dense(300, activation="relu"))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(300, return_sequences=True)))
    model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(1e-3))
    model.summary()
    return model

## 4.1 基本参数设置

In [ ]:
train_data_X.shape

In [58]:
# 输入的长度
input_length = train_data_X.shape[1]
# 输出的长度
output_sequence_length = train_data_Y.shape[1]
# 词表大小
vocab_size=len(vocab)
# 词向量矩阵
embedding_matrix = wv_model.wv.vectors

## 4.2 模型构建

In [59]:
model = seq2seq(input_length,output_sequence_length,embedding_matrix,vocab_size)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 204, 300)          9325500   
_________________________________________________________________
bidirectional (Bidirectional (None, 600)               1083600   
_________________________________________________________________
dense (Dense)                (None, 300)               180300    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 42, 300)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 42, 600)           1083600   
_________________________________________________________________
time_distributed (TimeDistri (None, 42, 31085)         18682085  
Total params: 30,355,085
Trainable params: 21,029,585
Non-trainable params: 9,325,500
____________________________________

## 4.3 模型训练

In [60]:
model.fit(train_data_X, train_data_Y, batch_size=32, epochs=1, validation_split=0.2)

2072/2072 [==============================] - 11114s 5s/step - loss: 2.8853 - val_loss: 2.6326


## 4.4 模型保存

In [ ]:
model.save('data/seq2seq_model.h')

## 4 .5 模型预测

In [1]:
test_data_Y = model.predict(test_data_X)

NameError: name 'model' is not defined

# seq2seq

1. 所有输出端，都以一个通用的<start>标记开头，以<end>标记结尾，这两个标记也视为一个词/字；

2. 将<start>输入decoder，然后得到隐藏层向量，将这个向量与encoder的输出混合，然后送入一个分类器，分类器的结果应当输出P；

3. 将P输入decoder，得到新的隐藏层向量，再次与encoder的输出混合，送入分类器，分类器应输出Q；

4. 依此递归，直到分类器的结果输出<end>。
    

* 回到用seq2seq生成文章标题这个任务上，模型可以做些简化，并且可以引入一些先验知识。比如，由于输入语言和输出语言都是中文，因此encoder和decoder的Embedding层可以共享参数（也就是用同一套词向量）。这使得模型的参数量大幅度减少了。


In [ ]:
from keras.preprocessing.text import Tokenizer # 词表构建 单词过滤 词频统计 序列填充
from keras.preprocessing.sequence import pad_sequences # 序列数据填充
from sklearn.model_selection import train_test_split # 数据集划分